<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/helicity_aware_hall_thruster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mhd_thruster_from_equations.py
# ------------------------------------------------------------
# Manuscript equations only (no external data).
# Saves four SEPARATE figures: figures/eqscan_chi.png, ..._jtheta.png, ..._fz.png, ..._corr.png

from dataclasses import dataclass
from typing import Tuple
import os
import numpy as np
import matplotlib.pyplot as plt

# ---------- User controls ----------
@dataclass
class Params:
    L: float = 0.10       # axial length [m]
    Nz: int = 1000        # grid points
    Ez: float = 150.0     # axial E-field [V/m]
    Br0: float = 0.02     # radial B scale [T]
    sigma_thetaz: float = 50.0  # [S/m]
    u0: float = 12000.0   # ion bulk speed [m/s]
    mode: str = "parametric"    # or "constant"

# ---------- Grid & profiles ----------
def grid(p: Params):
    z = np.linspace(0.0, p.L, p.Nz)
    dz = z[1] - z[0]
    return z, dz

def profiles(z: np.ndarray, p: Params) -> Tuple[np.ndarray, np.ndarray]:
    if p.mode.lower() == "constant":
        return np.full_like(z, p.u0), np.full_like(z, p.Br0)
    u  = p.u0  * (0.4 + 0.6 / (1.0 + np.exp(-(z/p.L - 0.6)*20.0)))
    Br = p.Br0 * (0.6 + 0.4*np.exp(-((z/p.L - 0.6)/0.12)**2))
    return u, Br

# ---------- Manuscript equations ----------
def chi(u, Br, Ez): return (u*Br)/Ez
def j_theta(chi_val, p): return p.sigma_thetaz * p.Ez * (1.0 - chi_val)
def fz(chi_val, Br, p):  return p.sigma_thetaz * p.Ez * Br * (chi_val - 1.0)

# ---------- Correlation helpers (Pearson-normalized) ----------
def standardize(x):
    mu = x.mean(); sd = x.std(ddof=1)
    return (x - mu) / (sd if sd > 0 else 1.0)

def lagged_cross_corr(H, u, max_lag):
    """C_HU[ell] = mean( H~_k * u~_{k+ell} ), ell >= 0."""
    Hs, us = standardize(H), standardize(u)
    if np.allclose(Hs, 0) or np.allclose(us, 0):  # flat signals
        return np.array([0]), np.array([0.0])
    N = len(Hs)
    max_lag = int(min(max_lag, N-2))
    lags = np.arange(0, max_lag+1, dtype=int)
    C = np.zeros_like(lags, float)
    for i, ell in enumerate(lags):
        C[i] = np.mean(Hs[:N-ell] * us[ell:])
    return lags, C

def parabolic_peak(C, lags):
    i0 = int(np.argmax(C))
    if i0 == 0 or i0 == len(C)-1:
        return float(lags[i0]), float(C[i0])
    Cm, C0, Cp = C[i0-1], C[i0], C[i0+1]
    denom = (Cm - 2*C0 + Cp)
    if abs(denom) < 1e-12:
        return float(lags[i0]), float(C0)
    delta = 0.5*(Cm - Cp)/denom
    Cref  = C0 - 0.25*(Cm - Cp)*delta
    return float(lags[i0] + delta), float(Cref)

# ---------- Safe figure saver ----------
def savefig(path, fig=None, dpi=150):
    if fig is None:
        fig = plt.gcf()
    fig.tight_layout()
    fig.savefig(path, dpi=dpi)
    plt.close(fig)

# ---------- Runner ----------
def main():
    outdir = "figures"
    os.makedirs(outdir, exist_ok=True)

    p = Params(mode="parametric")
    z, dz = grid(p)
    u, Br = profiles(z, p)

    chi_z = chi(u, Br, p.Ez)
    jt = j_theta(chi_z, p)
    fz_z = fz(chi_z, Br, p)

    # Integrated thrust (per unit cross-section)
    Fz_line = np.trapz(fz_z, z)
    frac_pos = np.mean(chi_z > 1.0)

    print("=== Manuscript-equation sweep (no external data) ===")
    print(f"E_z = {p.Ez:.2f} V/m,  sigma_theta_z = {p.sigma_thetaz:.2f} S/m,  u0 ~ {p.u0:.1f} m/s,  Br0 ~ {p.Br0:.4f} T")
    print(f"Length L = {p.L:.3f} m,  Nz = {p.Nz}")
    print(f"Integrated thrust (per unit cross-section)  ∫ f_z dz = {Fz_line:.3e} N/m^2")
    print(f"Fraction of channel with chi > 1: {100*frac_pos:.1f}%")

    # Pearson-normalized cross-correlation (positive lag ⇒ u lags H)
    H = u * Br
    lags, C = lagged_cross_corr(H, u, max_lag=200)
    if len(lags) > 1:
        lag_star_samp = lags[int(np.argmax(C))]
        lag_star_ref, C_star_ref = parabolic_peak(C, lags)
        Dz_star = lag_star_ref * dz
        print(f"Cross-corr peak (grid):    lag={lag_star_samp} samples  -> Δz* ~ {lag_star_samp*dz:.3e} m")
        print(f"Cross-corr peak (refined): lag={lag_star_ref:.2f} samples -> Δz* ~ {Dz_star:.3e} m,  C* ~ {C_star_ref:.3f}")
    else:
        print("Cross-correlation skipped (no variance in profiles).")

    # --------- Separate plots (each saved & closed) ---------
    fig1, ax1 = plt.subplots(figsize=(6, 3.6))
    ax1.plot(z, chi_z)
    ax1.axhline(1.0, linestyle="--")
    ax1.set_xlabel("z [m]"); ax1.set_ylabel("chi")
    ax1.set_title("Motional-field ratio χ = u B_r / E_z")
    savefig(os.path.join(outdir, "eqscan_chi.png"), fig1)

    fig2, ax2 = plt.subplots(figsize=(6, 3.6))
    ax2.plot(z, jt)
    ax2.set_xlabel("z [m]"); ax2.set_ylabel("j_theta [A/m^2]")
    ax2.set_title("Azimuthal current density j_theta")
    savefig(os.path.join(outdir, "eqscan_jtheta.png"), fig2)

    fig3, ax3 = plt.subplots(figsize=(6, 3.6))
    ax3.plot(z, fz_z)
    ax3.set_xlabel("z [m]"); ax3.set_ylabel("f_z [N/m^3]")
    ax3.set_title("Axial force density f_z")
    savefig(os.path.join(outdir, "eqscan_fz.png"), fig3)

    if len(lags) > 1:
        fig4, ax4 = plt.subplots(figsize=(6, 3.6))
        ax4.plot(lags*dz, C)
        ax4.set_xlabel("Δz [m]"); ax4.set_ylabel("Correlation")
        ax4.set_title("Lagged cross-correlation: H(z)=u B_r vs u(z)")
        if 'Dz_star' in locals():
            ax4.axvline(Dz_star, linestyle="--")
        savefig(os.path.join(outdir, "eqscan_corr.png"), fig4)

    print("Saved to:", os.path.abspath(outdir))

if __name__ == "__main__":
    main()


/tmp/ipython-input-21905418.py:93: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  Fz_line = np.trapz(fz_z, z)


=== Manuscript-equation sweep (no external data) ===
E_z = 150.00 V/m,  sigma_theta_z = 50.00 S/m,  u0 ~ 12000.0 m/s,  Br0 ~ 0.0200 T
Length L = 0.100 m,  Nz = 1000
Integrated thrust (per unit cross-section)  ∫ f_z dz = -2.627e+00 N/m^2
Fraction of channel with chi > 1: 21.2%
Cross-corr peak (grid):    lag=88 samples  -> Δz* ~ 8.809e-03 m
Cross-corr peak (refined): lag=88.10 samples -> Δz* ~ 8.818e-03 m,  C* ~ 0.979
Saved to: /content/figures
